- Pipelines are a simple way to group data preprocessing and modelling steps together and keep them organized.
- They bundle these steps so that the whole bundle can be used as a single step.

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("melb_data.csv")

y = data.Price
X = data.drop(['Price'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=0)

cateCols = [col for col in X_train.columns if X_train[col].nunique() < 10 and X_train[col].dtype == "object"]

numeCols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

final_cols = cateCols + numeCols
X_train_final = X_train[final_cols].copy()
X_valid_final = X_valid[final_cols].copy()

X_train_final.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


First step in the pipeline is preprocessing, for whcih we will use the `ColumnTransformer` class to bundle together different preprocessing steps. We want to carry out the following preprocessing steps:
- Numerical data: Impute missing values
- Categorical data: Imputer missing values and apply one-hot encoding

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# For numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# For categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeCols),
        ('cat', categorical_transformer, cateCols)
    ]
)

In [3]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

Pipelines can contain ColumnTransformers which can contain Pipelines too.
Now, we can have a main Pipeline that bundles the preprocessing and modelling steps:

In [4]:
from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

my_pipeline.fit(X_train, y_train)
predictions = my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid, predictions)
print(score)

160679.18917034855
